In [5]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os, re
data_path = "C:/Users/SpiffyApple/Documents/USC/OwnResearch/marketShare"

import matplotlib.pyplot as plt
%matplotlib inline
jet = plt.get_cmap('jet')

import geopandas as gpd
from geopandas.tools import sjoin
from shapely.geometry import Point

import itertools

data_path = 'C:/Users/SpiffyApple/Documents/USC/OwnResearch/marketShare'

# Purpose

The purpose of this notebook is to merge NC parcels with block group TIGER data

The goal is to then merge the files using the address fields

## Parcel Data

In [6]:
# load parcel data
parcels = gpd.read_file("/".join([data_path, 'wake_parcels','nc_wake_parcels_pt.shp']))

In [27]:
# verify the data are for Wake County
parcels.loc[:,'CNTYNAME'].unique()

array(['Wake'], dtype=object)

In [41]:
# check owner name against something matchable in the assessor dataset
parcels.loc[parcels.OWNNAME.str.contains("COLEMAN GROUP INC"),['MAILADD','MCITY']]

,MAILADD,MCITY
38535,115 S SAINT MARYS ST,RALEIGH NC 27603-1699
44408,115 S SAINT MARYS ST,RALEIGH NC 27603-1699
62831,115 S SAINT MARYS ST,RALEIGH NC 27603-1699
94784,115 S SAINT MARYS ST,RALEIGH NC 27603-1699
249311,115 S SAINT MARYS ST,RALEIGH NC 27603-1699


In [15]:
# load TIGER file
bg = gpd.read_file("/".join([data_path, 'tl_2010_37183_bg10','tl_2010_37183_bg10.shp']))

In [38]:
# drop parcels without an owner
parcels = parcels[parcels.OWNNAME.notnull()]

In [42]:
# keep a select number of features
cols2keep = ['ALTPARNO','PARNO','PARUSECODE','PARUSEDESC','OWNNAME','MAILADD','MCITY','geometry']
parcels = parcels.loc[:,cols2keep]

In [57]:
#parcels.to_crs('epsg:4269')

In [49]:
parcels.crs

{'proj': 'lcc',
 'lat_1': 34.33333333333334,
 'lat_2': 36.16666666666666,
 'lat_0': 33.75,
 'lon_0': -79,
 'x_0': 609601.2199999999,
 'y_0': 0,
 'datum': 'NAD83',
 'units': 'us-ft',
 'no_defs': True}

In [50]:
# save parcel data to then re-project in QGIS to epsg4269
# have to do this because geopandas' reprojection doesn't work
parcels.to_file("/".join([data_path, 'parcels_ver0.shp']))

## TIGER data

In [43]:
bg = gpd.read_file("/".join([data_path, 'tl_2010_37183_bg10','tl_2010_37183_bg10.shp']))

In [46]:
cols2drop = ['NAMELSAD10', 'MTFCC10', 'FUNCSTAT10', 'ALAND10', 'AWATER10',
       'INTPTLAT10', 'INTPTLON10']
bg.drop(cols2drop,axis=1,inplace=True)

# Combine the two data sets

In [56]:
pr = gpd.read_file("/".join([data_path, 'wake_parcels', 'parcels_ver1.shp']))

In [58]:
df = gpd.sjoin(pr,bg,how='left')

In [66]:
print("Is merged same size as parcels data: %s" %(df.shape[0] == pr.shape[0]))

Is merged same size as parcels data: True


In [67]:
i = bg.GEOID10.unique().shape[0]
j = df.GEOID10.unique().shape[0]
print("Number of block groups in TIGER and merged files: %d and %d respectively" %(i,j))

Number of block groups in TIGER and merged files: 455 and 454 respectively


In [68]:
df.drop(['geometry'],axis=1).to_csv("/".join([data_path, 'geocoded_wake_parcels.csv']))